# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


1. [Обзор данных](#1)
2. [Заполнение пропусков](#2)  
    2.2 [Проверка данных на аномалии и исправления](#3)  
    2.3 [Изменение типов данных](#4)  
    2.4 [Удаление дубликатов](#5)  
    2.5 [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#6)  
    2.6 [Категоризация дохода](#7)  
    2.7[Категоризация целей кредита](#8)  
3.[Ответы на вопросы](#9)  
    3.1 [Вопрос 1](#10)  
    3.2 [Вывод 1](#11)  
    3.3 [Вопрос 2](#12)  
    3.4 [Вывод 2](#13)  
    3.5 [Вопрос 3](#14)  
    3.6 [Вывод 3](#15)  
    3.7 [Вопрос 4](#16)  
    3.8 [Вывод 4](#16)  
4.[Общий вывод](#17)  



### Шаг 1. Обзор данных
<a id="1"></a>

In [1]:
# импорт Pandas, создание датафрейма, вывод 10 строк датафрейма на экран 
import pandas as pd

df = pd.read_csv('/datasets/data.csv')

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# получение информации о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



### Шаг 2.1 Заполнение пропусков
<a id="2"></a>

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом будут обработаны на следующем этапе. Найдите другой столбец и заполните пропущенные значения в нём медианным значением:

   - опишите, какие пропущенные значения вы обнаружили;
   - проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;
   - приведите возможные причины появления пропусков в данных;
   - объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

In [3]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски встречаются в колнке total_income и days_employed

In [4]:
#вычисляем долю пропущенных значений в столбце days_employed
missed_days_share = df['days_employed'].isna().sum() / df.shape[0]

#вычисляем долю пропущенных значений в столбце total_income
missed_income_share = df['total_income'].isna().sum() / df.shape[0]


print('Доля пропусков в столбце days_employed составляет {:.0%}'.format(missed_days_share))
print('Доля пропусков в столбце total_income составляет {:.0%}'.format(missed_income_share))



Доля пропусков в столбце days_employed составляет 10%
Доля пропусков в столбце total_income составляет 10%



**ВЫВОД**  

Возможная причина появления пропусков  - отсутствие трудового стажа, а так же возможный выход на пенсию или в отпуск по уходу за ребенком


In [5]:
#заполняем пропуски в столбце total_income медианным значением 
median_income = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(median_income)

Медиана является более надежным показателем при анализе зарплат, поскольку она устраняет проблему искажения данных, которая может возникнуть при использовании среднего значения. Когда некоторые значения значительно отличаются от большинства, такие как очень низкие или очень высокие доходы, медиана позволяет получить более объективное представление о распределении доходов в выборке.

In [6]:
#проверяем результат заполнения
df['total_income'].isna().sum()

0

### Шаг 2.2 Проверка данных на аномалии и исправления.
<a id="3"></a>

In [7]:
#заменяем отрицательные значения в столбце days_employed на модули
df['days_employed'] = df['days_employed'].abs()

#заполняем пропуски медианными значениями 
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

#проверяем замены
df['days_employed'].isna().sum()


0

In [8]:
#ищем дубликаты в других столбцах
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [9]:
#приводим значения к единому формату
df['family_status'] = df['family_status'].replace('Не женат / не замужем', 'не женат / не замужем')
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [10]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [11]:
#убираем дубликаты
df['education'] = df['education'].str.lower()

In [12]:
#сколько детей у заемщиков
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значения -1 и 20 заменим на 1 и 2, так как статистически не может быть такого количества семей с 20 детьми. Это скорее исключение а не правило.

In [13]:
#замена значения -1 а так же 20 для количества детей в семьях
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [14]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Появился странный гендер XNA

In [15]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [16]:
#заменим ошибочный гендер на мужской
df.loc[df['gender'] == 'XNA','gender'] = 'M'
df['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

### Шаг 2.3. Изменение типов данных.
<a id="4"></a>

In [17]:
#заменяем вещественный тип числа в total_income на целочисленный
df['total_income'] = df['total_income'].astype(int)

### Шаг 2.4. Удаление дубликатов.
<a id="5"></a>

In [21]:
#находим полностью дублирующиеся строки
df.duplicated().sum()

71

In [23]:
#удаляем дубликаты, и проверяем наличие
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

метод поиска - перебор всех колонок, и выявление дубликатов. Основной причниой поиска дубликатов вижу человеческий фактор.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
<a id="6"></a>

In [24]:
#создаём словари
education_dict = df[['education_id', 'education']]
family_status_dict = df[['family_status_id','family_status']]

#удаляем дубликаты о сортируем в порядке возрастания индексов
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True)

#удаляем столбцы из датафрейма
df = df.drop(columns = ['education'],axis = 1) 
df = df.drop(columns = ['family_status'],axis = 1) 

### Шаг 2.6. Категоризация дохода.
<a id="7"></a>

In [25]:
#функция для присвоения категорий в соответствии с доходом

def income_category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

#добавим столбец total_income_category в датафрейм
df['total_income_category'] = df['total_income'].apply(income_category)


### Шаг 2.7. Категоризация целей кредита.
<a id="8"></a>

In [26]:
# какие цели кредита есть в датафрейме
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [27]:
# функция, которая на основании данных из столбца purpose сформирует новый столбец purpose_category
def purpose(purpose_category):
    if 'свадьб' in purpose_category:
        return 'проведение свадьбы'
    if 'жиль' in purpose_category or 'недвижим' in purpose_category:
        return 'операции с недвижимостью'
    if 'автомоб' in purpose_category:
        return 'операции с автомобилем'
    if 'образован' in  purpose_category:
        return 'получение образования'

    
# новый столбец в датафрейм
df['purpose_category'] = df['purpose'].apply(purpose)

# всем ли значениям присвоились категории
df['purpose_category'].value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

In [28]:
#выведем первые 10 столбцов обновленного датафрейма на экрам
df.head(10)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.
<a id="9"></a>

##### Вопрос 1:
<a id="10"></a>
Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [29]:
#построим сводную таблицу для проверки зависимости 
children_pivot = df.pivot_table(index='children', columns='debt', values='days_employed', aggfunc='count')

#определим долю невозвратных кредитов относительно всех
children_pivot['share'] = children_pivot[1] / children_pivot[0]
children_pivot.sort_values(by = 'share')

debt,0,1,share
children,,,
0,13028.0,1063.0,0.081593
3,303.0,27.0,0.089109
1,4410.0,445.0,0.100907
2,1926.0,202.0,0.104881
4,37.0,4.0,0.108108
5,9.0,NaN,NaN


##### Вывод 1:
<a id="11"></a>

Заёмщики без детей возвращают кредит чаще чем заёмщики с детьми

##### Вопрос 2:
<a id="12"></a>
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
#построим сводную таблицу для проверки зависимости 
family_pivot = df.pivot_table(index=['gender','family_status_id'], columns='debt', values='days_employed', aggfunc='count')
family_pivot['share'] = family_pivot[1] / family_pivot[0]
family_pivot = family_pivot.reset_index()
family_pivot = family_pivot.merge(family_status_dict, on = 'family_status_id', how = 'left')
family_pivot.sort_values(by = 'share')

,gender,family_status_id,0,1,share,family_status
2,F,2,852,52,0.061033,вдовец / вдова
3,F,3,875,61,0.069714,в разводе
4,F,4,1611,118,0.073246,не женат / не замужем
0,F,0,7230,530,0.073306,женат / замужем
1,F,1,2612,233,0.089204,гражданский брак
5,M,0,4178,401,0.095979,женат / замужем
8,M,3,235,24,0.102128,в разводе
6,M,1,1151,155,0.134666,гражданский брак
9,M,4,925,156,0.168649,не женат / не замужем
7,M,2,44,11,0.250000,вдовец / вдова


##### Вывод 2:
<a id="13"></a>
Самая надёжная категория заёмщиков  - вдовы, следом идут разведенные женщины.Женщины возвращают кредит лучше чем мужчины. Самые ненадёжные заёмщики  - вдовцы.

##### Вопрос 3:
<a id="14"></a>
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
#построим сводную таблицу для проверки зависимости 
income_pivot = df.pivot_table(index=['gender','total_income_category'], columns='debt', values='days_employed', aggfunc='count')
income_pivot['share'] = income_pivot[1] / income_pivot[0]
income_pivot = income_pivot.reset_index()
income_pivot.sort_values(by = 'share')

debt,gender,total_income_category,0,1,share
3,F,D,294.0,15.0,0.051020
1,F,B,2547.0,164.0,0.064389
2,F,C,10315.0,813.0,0.078817
6,M,B,2138.0,192.0,0.089804
7,M,C,4341.0,547.0,0.126008
4,F,E,14.0,2.0,0.142857
5,M,A,13.0,2.0,0.153846
8,M,D,35.0,6.0,0.171429
0,F,A,10.0,NaN,NaN
9,M,E,6.0,NaN,NaN


##### Вывод 3:
<a id="15"></a>
самые надежные заемщики  - мужчины с доходом ниже 30к и женщины с доходом от 1кк а так же  женищины с доходами 30к-50к и женщины с доходом 200к - 1кк.
Самые ненадежные заемщики - мужчины с доходами 30к-50к и мужчины с доходами больше 1кк 

##### Вопрос 4:
<a id="16"></a>
Как разные цели кредита влияют на его возврат в срок?

In [32]:
purpose_pivot = df.pivot_table(index=['gender','purpose_category'], columns='debt', values='days_employed', aggfunc='count')
purpose_pivot['share'] = purpose_pivot[1] / purpose_pivot[0]
purpose_pivot = purpose_pivot.reset_index()
purpose_pivot.sort_values(by = 'share')

debt,gender,purpose_category,0,1,share
1,F,операции с недвижимостью,6641,444,0.066857
3,F,проведение свадьбы,1454,108,0.074278
2,F,получение образования,2460,207,0.084146
0,F,операции с автомобилем,2625,235,0.089524
5,M,операции с недвижимостью,3388,338,0.099764
7,M,проведение свадьбы,684,78,0.114035
4,M,операции с автомобилем,1278,168,0.131455
6,M,получение образования,1183,163,0.137785


##### Вывод 4:
<a id="17"></a>

самые надежные заемщики  - женщины, кредиты на все цели они возвращают лучше мужчин.
хуже всего отдают кредит мужчины на получение образования. 


## Общий вывод:
<a id="18"></a>

в качестве общего вывода предлагаю найти самых надежных и самых ненажедных заёмщиков

In [33]:
total_pivot = df.pivot_table(index=['gender','purpose_category','total_income_category','family_status_id','children','education_id'], columns='debt', values='days_employed', aggfunc='count')
total_pivot = total_pivot.reset_index()
total_pivot['share'] = total_pivot[1] / total_pivot[0]
total_pivot = total_pivot.sort_values(by = 'share')
total_pivot = total_pivot.merge(family_status_dict, on = 'family_status_id', how = 'left')
#total_pivot = total_pivot.merge(education_dict, on = 'education_id', how = 'left')
total_pivot.tail(428)

,gender,purpose_category,total_income_category,family_status_id,children,education_id,0,1,share,family_status
325,M,получение образования,C,1,1,1,4.0,5.0,1.25,гражданский брак
326,M,операции с автомобилем,C,4,2,1,1.0,2.0,2.00,не женат / не замужем
327,M,операции с недвижимостью,C,1,0,3,1.0,2.0,2.00,гражданский брак
328,F,операции с автомобилем,A,0,1,0,1.0,NaN,NaN,женат / замужем
329,F,операции с автомобилем,A,4,1,0,1.0,NaN,NaN,не женат / не замужем
...,...,...,...,...,...,...,...,...,...,...
748,M,проведение свадьбы,C,1,4,0,1.0,NaN,NaN,гражданский брак
749,M,проведение свадьбы,D,1,0,2,1.0,NaN,NaN,гражданский брак
750,M,проведение свадьбы,D,1,0,3,1.0,NaN,NaN,гражданский брак
751,M,проведение свадьбы,D,1,1,3,1.0,NaN,NaN,гражданский брак


Портрет надежного заемщика выглоядит так - мужчина, с любым уровнем дохода, состоящий в гражданском браке, с детьми, который берет кредит на проведение свадьбы. Задолжености по таким кредитам и крадиторам отсутствуют

Портрет хучшего заёмщика выглядит так - мужчина без детей, с начальным образованием, состоящий в гражданском браке, который берет кредит на операции с недвижимостью